# Simple GLPK problems

In [1]:
using JuMP, GLPK#, Ipopt

A=[1.0 1.0 0.0;1.0 0.0 1.0]
b=[2.0;2.0]
c=[1.0;2.0;3.0];

## Problema lineal

In [2]:
model = JuMP.Model(GLPK.Optimizer)
@variable(model, x[1:3]>=0)
@constraint(model, A*x.<=b)
@objective(model, Max, c'*x)

print(model)

optimize!(model)
value.(x), objective_value(model)

([0.0, 2.0, 2.0], 10.0)

## Problema lineal con variables binarias (0 o 1)

In [3]:
## AHora la versión binaria. Debería dar menos por ser solo 0 o 1

model = JuMP.Model(GLPK.Optimizer)

@variable(model, x[1:3]>=0, Bin)
@constraint(model, A*x.<=b)
@objective(model, Max, c'*x)

print(model)

optimize!(model)
value.(x), objective_value(model)

([1.0, 1.0, 1.0], 6.0)

## Problema lineal con variables enteras

In [4]:
## AHora la versión entera. Debería dar lo mismo que el primero que dio entero.

model = JuMP.Model(GLPK.Optimizer)

@variable(model, x[1:3]>=0, Int)
@constraint(model, A*x.<=b)
@objective(model, Max, c'*x)

print(model)

optimize!(model)
value.(x), objective_value(model)

([0.0, 2.0, 2.0], 10.0)

## Location selection

### Ejemplo 1

In [5]:
function get_cost_matrix(m, n, x, y)
    #m=length(x)
    #n=length(y)
    #return abs.(x*ones(1,n) - ones(m,1)*y') 
    
    #Calculo los costos Cij = |xi-yj| (distancia euclidea en R^2)
    w1 = (x[:,1]*ones(1,n) - ones(m,1)*y[:,1]').^2
    w2 = (x[:,2]*ones(1,n) - ones(m,1)*y[:,2]').^2
    return (w1+w2).^(1/2)
end

get_cost_matrix (generic function with 1 method)

In [6]:
m = 100 # Cantidad de EVs
n = 10; # Cantidad de cargadores

In [7]:
#x = [1; 2; 5]
x = sortslices(rand(m,2), dims = 1);
#y = [1; 2; 5]
y = sortslices(rand(n,2), dims = 1);
c = get_cost_matrix(m, n, x, y); #Matriz de costos

In [8]:
smax = 100*ones(n,1) # Capacidad máxima de cargadores
k = 3 # Cantidad de estaciones prendidas
q = 1*ones(m,1); # Demandas

In [9]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:m,1:n]>=0)
@variable(model, b[1:n], Bin)

@constraint(model, sum(pi, dims=2).== q)
@constraint(model, sum(pi, dims=1)' .<= (b.*smax) )
@constraint(model, sum(b) <= k)

@objective(model,Min, sum( c.*pi ))

#print(model)
model

A JuMP Model
Minimization problem with:
Variables: 1010
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 100 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 11 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1000 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 10 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: b, pi

In [10]:
optimize!(model)
value.(pi), objective_value(model)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 1.0 0.0], 27.278353011912728)

In [11]:
value.(pi)

100×10 Matrix{Float64}:
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0

In [12]:
value.(b)

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0

### Fixed capacities

In [11]:
c = [1 3; 3 1] # Costo
s = [0 2] # Capacidad de cargadores (en autos)
q = [1; 1]; # Demandas (cargo o no cargo)

In [12]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:2,1:2]>=0)

@constraint(model, sum(pi,dims=2).== q)
@constraint(model, sum(pi, dims=1)' .== s )

@objective(model,Min, sum( c.*pi ))

print(model)

optimize!(model)
value.(pi), objective_value(model)

([0.0 1.0; 0.0 -1.0], 2.0)

In [13]:
value.(pi)

2×2 Matrix{Float64}:
 0.0   1.0
 0.0  -1.0

## Constrained or penalized capacities

In [14]:
c = [1 3; 3 1] # Costo
smax = [2 2] # Capacidad de cargadores (en autos)
q = [1; 1]; # Demandas (cargo o no cargo)

In [15]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model,pi[1:2,1:2]>=0)

@constraint(model, sum(pi,dims=2).== q)
@constraint(model, sum(pi, dims=1)' .<= smax )

@objective(model,Min, sum( c.*pi ))

print(model)

optimize!(model)
value.(pi), objective_value(model)

([1.0 0.0; 0.0 1.0], 2.0)

In [16]:
value.(pi)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

### Penalized

In [17]:
function phi(s)
    return 0.5*(s.^2)
end

phi (generic function with 1 method)

In [18]:
c = [1 3; 3 1] # Costo
smax = [2 2] # Capacidad de cargadores (en autos)
q = [1; 1]; # Demandas (cargo o no cargo)
eps = 0.1;

In [19]:
#model = JuMP.Model(GLPK.Optimizer)
model = JuMP.Model(Ipopt.Optimizer)

@variable(model, pi[1:2,1:2]>=0)
@variable(model, s[1:2]>=0)

@constraint(model, sum(pi,dims=2).== q)
@constraint(model, sum(pi, dims=1)' .== s )

@objective(model,Min, sum( c.*pi ) + eps*sum(phi(s)))

print(model)

optimize!(model)
value.(pi), objective_value(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       10
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality co

([1.0000000087461995 -8.746199604432545e-9; -8.746199604432545e-9 1.0000000087461995], 2.099999965015201)

In [20]:
value.(pi)

2×2 Matrix{Float64}:
  1.0        -8.7462e-9
 -8.7462e-9   1.0